In [11]:
import chromedriver_autoinstaller
import time
from selenium import webdriver
from bs4  import BeautifulSoup as bs
import csv
from selenium.webdriver.common.by import By
import requests
import urllib.request
from tqdm import tqdm 
import pandas as pd

In [2]:
url = "https://nid.naver.com/nidlogin.login"
id = 'Your ID'
pw = 'Your Password'

In [3]:
dir_driver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
driver = webdriver.Chrome('./chromedriver.exe')
driver.get(url)

driver.implicitly_wait(2)

C:\Users\user\anaconda3\envs\crawling\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [4]:
driver.execute_script("document.getElementsByName('id')[0].value = '" + id + "'")
driver.execute_script("document.getElementsByName('pw')[0].value = '" + pw + "'")

In [5]:
button = driver.find_element(By.ID, "log.login")
button.click()
time.sleep(1)

In [6]:
baseurl = 'https://cafe.naver.com/koreanmania'
driver.get(baseurl)
time.sleep(1)

In [7]:
menu = driver.find_element(By.ID, "menuLink24")
menu.click()
time.sleep(1)

In [8]:
driver.switch_to.frame("cafe_main")
time.sleep(1)

In [9]:
soup = bs(driver.page_source ,'html.parser')
soup = soup.find_all(class_ = 'article-board m-tcol-c')[1]

In [10]:
page_num = 13
clubid = 11276433
menuid = 24
title_list = []
url_list = []
for page in tqdm(range(1, page_num+1)):
    driver.find_element_by_link_text(str(page)).click()
    time.sleep(1)
    soup = bs(driver.page_source ,'html.parser')
    soup = soup.find_all(class_ = 'article-board m-tcol-c')[1]
    datas = soup.find_all("td", class_ = 'td_article')
    for data in datas:
        article_title = data.find(class_ = 'article')
        link = article_title.get('href')
        article_title = article_title.get_text().strip()
        title_list.append(article_title)
        url_list.append(baseurl+link)
    if page % 10 == 0:
        driver.find_element_by_link_text(str('다음')).click()
        time.sleep(1)
    time.sleep(0.1)

  0%|                                                                                           | 0/13 [00:00<?, ?it/s]C:\Users\user\anaconda3\envs\crawling\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  import sys
100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:19<00:00,  1.46s/it]


In [12]:
df = pd.DataFrame((zip(title_list, url_list)), columns = ['Title', 'URL'])
df.head()

,Title,URL
0,그 카페스탭한테.........,https://cafe.naver.com/koreanmania/ArticleRead...
1,정신이 뿌연 느낌? 약이문제일까요?,https://cafe.naver.com/koreanmania/ArticleRead...
2,스피닝 운동,https://cafe.naver.com/koreanmania/ArticleRead...
3,.......,https://cafe.naver.com/koreanmania/ArticleRead...
4,우울증 제발 도와주세요..(우울한글 우울하면보지말아주세요),https://cafe.naver.com/koreanmania/ArticleRead...


In [13]:
contents = []
for i,url in enumerate(tqdm(url_list)):
    try:
        driver.get(url)
        time.sleep(0.5)
        driver.switch_to.frame('cafe_main')
        soup = bs(driver.page_source, 'html.parser')
        content_tags = soup.select('#tbody')[0].select('p')
        content = ' '.join([ tags.get_text() for tags in content_tags ])
        contents.append(content)
        time.sleep(1)
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 195/195 [06:56<00:00,  2.14s/it]


In [15]:
print(contents[1])

잠이 안깬것처럼 깊은사고도 안되는거같고 ​ ​ 에프람정 20밀리그램 아빌리파이정 2밀리그램 독세정 6밀리그램 ​ 한미알마게이트정 ​ 이렇게먹는데 ​ 약은 괜찮은거같기도하구요 ​ 아빌리파이가 살찐다해서 안먹었다가 너무 무기력한거같아서 그냥 먹고있어요 ​ 귀찮아서 약안먹는때도 많구요.. 꾸준히먹어서 괜찮아졌음좋겠네요 ​
